## Melanoma(skin cancer) Classification

Skin cancer is the most prevalent type of cancer. Melanoma, specifically, is responsible for 75% of skin cancer deaths, despite being the least common skin cancer.It has an ability to spread to other organs more rapidly if it is not treated at an early stage.
The American Cancer Society estimates over 100,000 new melanoma cases will be diagnosed in 2020. It's also expected that almost 7,000 people will die from the disease. As with other cancers, early and accurate detection—potentially aided by data science—can make treatment more effective.

In this competition given an image of the cancer we are asked to predict whether it's beingn or malignant.

So let's get started.

In [ ]:
import tensorflow
print(tensorflow.__version__)

In [ ]:
!pip install livelossplot

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import PIL
from IPython.display import Image, display
from keras.applications.vgg16 import VGG16,preprocess_input
# Plotly for the interactive viewer (see last section)
import plotly.graph_objs as go
import plotly.graph_objects as go
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model,load_model
from keras.applications.vgg16 import VGG16,preprocess_input
from keras.applications.resnet50 import ResNet50
from keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten,BatchNormalization,Activation
from keras.layers import GlobalMaxPooling2D
from keras.models import Model
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import gc
import skimage.io
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.python.keras import backend as K
from livelossplot import PlotLossesKeras

I'm gonna be using the jpeg files for training and testing.

In [ ]:
train_dir='/kaggle/input/siim-isic-melanoma-classification/jpeg/train/'
test_dir='/kaggle/input/siim-isic-melanoma-classification/jpeg/test/'
train=pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/train.csv')
test=pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/test.csv')
submission=pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/sample_submission.csv')

In [ ]:
train.head()

Since this is medical data I'm expecting it to be unbalanced.

In [ ]:
train['target'].value_counts()

In [ ]:
dist=train['target'].value_counts()
print("Benign cases are",(32542/(32542+584))*100)


The difference is huge and only 1.7% patients in our data have malignant cancer.

**anatom_site_general_challenge** in the dataset refers to the location of the skin cancer given in the image.

In [ ]:
labels=train['anatom_site_general_challenge'].value_counts().index
values=train['anatom_site_general_challenge'].value_counts().values
print(labels, values)
fig = go.Figure(data=[go.Pie(labels=labels, values=values, textinfo='label+percent',
                             insidetextorientation='radial'
                            )])
fig.show()


In more than half of the patients in our dataset, the cancer is found on the torso.

Now if we look at the diagnosis provided by Dermatologists.(I have removed cases marked "unknown")

In [ ]:
labels=train['diagnosis'].value_counts().index[1:]
values=train['diagnosis'].value_counts().values[1:]
fig = go.Figure(data=[go.Pie(labels=labels, values=values, textinfo='label+percent',
                             insidetextorientation='radial'
                            )])
fig.show()


A "nevus" is basically a visible, circumscribed, chronic lesion of the skin. Since they are also called moles and also cover majority of the data, I think this diagnosis is for benign cases.

Let's check it out.

In [ ]:
new=train.drop(labels=['image_name','patient_id','sex','age_approx','anatom_site_general_challenge','target'],axis=1)
#print(new['diagnosis'].values,"\n",new['benign_malignant'])
pd.crosstab(new['diagnosis'].values,new['benign_malignant'])

> So my presumption was true and most benign cases are diagnosed as **nevus**. 

> All patients diagnosed as "melanoma" have malignant cancers. I think this term is only reserved for severe cases.

As I have already said the data is very imbalanced so I'll train on only a sample of it.

In [ ]:
df_0=train[train['target']==0].sample(2000)
df_1=train[train['target']==1]
train=pd.concat([df_0,df_1])
train=train.reset_index()

Before going any further with training let's take a look at sample photos from both classes.

In [ ]:
print('Benign Cases')
benign=[]
df_benign=df_0.sample(40)
df_benign=df_benign.reset_index()
for i in range(40):
    img=cv2.imread(str(train_dir + df_benign['image_name'].iloc[i]+'.jpg'))
    img = cv2.resize(img, (224,224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)/255.
    benign.append(img)
f, ax = plt.subplots(5,8, figsize=(10,8))
for i, img in enumerate(benign):
        ax[i//8, i%8].imshow(img)
        ax[i//8, i%8].axis('off')
        
plt.show()

In [ ]:
print('Malignant Cases')
malignant=[]
df_malignant=df_1.sample(40)
df_malignant=df_malignant.reset_index()
for i in range(40):
    img=cv2.imread(str(train_dir + df_malignant['image_name'].iloc[i]+'.jpg'))
    img = cv2.resize(img, (224,224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)/255.
    malignant.append(img)
f, ax = plt.subplots(5,8, figsize=(10,8))
for i, img in enumerate(malignant):
        ax[i//8, i%8].imshow(img)
        ax[i//8, i%8].axis('off')
        
plt.show()

## Preparing the Datasets

In [ ]:
labels=[]
data=[]
for i in range(train.shape[0]):
    data.append(train_dir + train['image_name'].iloc[i]+'.jpg')
    labels.append(train['target'].iloc[i])
df=pd.DataFrame(data)
df.columns=['images']
df['target']=labels
print(df.shape)
print(df.head())

In [ ]:
test_data=[]
for i in range(test.shape[0]):
    test_data.append(test_dir + test['image_name'].iloc[i]+'.jpg')
df_test=pd.DataFrame(test_data)
df_test.columns=['images']
print(df_test.shape)
print(df_test.head())

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(df['images'],df['target'], test_size=0.2, random_state=1234)

train=pd.DataFrame(X_train)
train.columns=['images']
train['target']=y_train

validation=pd.DataFrame(X_val)
validation.columns=['images']
validation['target']=y_val

print(train.shape)
print(train.head())
print(validation.shape)
print(validation.head())

I'll do some very basic preprocessing like 
* normalizing
* reshaping
* augmentation(only for tarin data)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,horizontal_flip=True, brightness_range=[0.5,1.5])
val_datagen=ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_dataframe(
    train,
    x_col='images',
    y_col='target',
    target_size=(224, 224),
    batch_size=8,
    shuffle=True,
    class_mode='raw')

validation_generator = val_datagen.flow_from_dataframe(
    validation,
    x_col='images',
    y_col='target',
    target_size=(224, 224),
    shuffle=False,
    batch_size=8,
    class_mode='raw')



## Modelling
I'm using pretrained VGG-16 and adding the last dense layer.
The competition is evaluated on AUC scores, so we'll use that as a metric.

In [ ]:
def vgg16_model( num_classes=None):

    model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    # Adding a one by one convolution to reduce the number of channels
    # BatchNormalization is applied to z not a. Thats why activaion function is applied after BatchNormalization
    x = Conv2D(filters = 64, kernel_size = (1, 1))(model.output)
    x = BatchNormalization(axis = 3)(x)
    x = Activation('relu')(x)
    
    x = Flatten()(x)
    
    #adding extra dense layer
    x = Dense(32, activation='relu')(x)
    x = Dropout(.5)(x)
    output = Dense(1,activation='sigmoid')(x) # because we have to predict the AUC
    model = Model(model.input,output)
    
    return model

vgg_conv=vgg16_model(1)

Also, because of class imbalance it's better to use **focal loss** rather than normal **binary_crossentropy**.You can read more about it [here](https://arxiv.org/abs/1708.02002)

In [ ]:
def focal_loss(alpha=0.25,gamma=2.0):
    def focal_crossentropy(y_true, y_pred):
        bce = K.binary_crossentropy(y_true, y_pred)
        
        y_pred = K.clip(y_pred, K.epsilon(), 1.- K.epsilon())
        p_t = (y_true*y_pred) + ((1-y_true)*(1-y_pred))
        
        alpha_factor = 1
        modulating_factor = 1

        alpha_factor = y_true*alpha + ((1-alpha)*(1-y_true))
        modulating_factor = K.pow((1-p_t), gamma)

        # compute the final loss and return
        return K.mean(alpha_factor*modulating_factor*bce, axis=-1)
    return focal_crossentropy

In [ ]:
opt = Adam(lr=1e-5)
vgg_conv.compile(loss=focal_loss(), metrics=[tf.keras.metrics.AUC()],optimizer=opt)

In [ ]:
nb_epochs = 25
batch_size= 8
nb_train_steps = train.shape[0]//batch_size
nb_val_steps=validation.shape[0]//batch_size
print("Number of training and validation steps: {} and {}".format(nb_train_steps,nb_val_steps))

In [ ]:
# DECREASE LEARNING RATE EACH EPOCH
annealer = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-5 * 0.95 ** epoch, verbose=1)

cb=[PlotLossesKeras(), annealer]
vgg_conv.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_steps,
    epochs=nb_epochs,
    validation_data=validation_generator,
    callbacks=cb,
    validation_steps=nb_val_steps)

## Submission

In [ ]:
def predict_with_TTA(test_gen, img, model):
    no_of_copies = 8
    samples = np.repeat(img, no_of_copies, axis=0)
    iterator = test_gen.flow(samples, batch_size = no_of_copies)
    predictions = model.predict(iterator, verbose=1)
#     f = model.predict(img, verbose=1)
#     print(np.mean(predictions))
#     print(f)
#     print(0.6 * f + np.mean(predictions) * 0.4)
    return np.mean(predictions)

In [ ]:
test_gen = ImageDataGenerator(rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,horizontal_flip=True)


target=[]
dft = df_test['images'].sample(20)
for path in dft:
    img=cv2.imread(str(path))
    img = cv2.resize(img, (224,224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)/255.
    img=np.reshape(img,(1,224,224,3))
    prediction=predict_with_TTA(test_gen, img, vgg_conv)
    target.append(prediction)
    
submission['target']=target
    
        

In [ ]:
submission.to_csv('submission.csv', index=False)
submission.head()

I'll keep on updating this kernel with new experiments.

If you liked it please upvote the kernel.